In [182]:
!pip install numpy
!pip install matplotlib
!pip install PIL


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [183]:
from matplotlib.image import imread
import numpy as np
from PIL import Image

In [184]:
image = imread('data/img_V2.png')[:, :, 0:3]
image.shape

(690, 500, 3)

In [185]:
# convert image to 8 bit per channel
image = (image * 255).astype(np.uint8)

image

array([[[243, 205,  96],
        [192, 152,  31],
        [201, 156,  11],
        ...,
        [253, 253, 253],
        [255, 255, 255],
        [233, 233, 233]],

       [[242, 204,  95],
        [191, 151,  30],
        [200, 155,  10],
        ...,
        [253, 253, 253],
        [255, 255, 255],
        [229, 229, 229]],

       [[242, 204,  95],
        [191, 151,  30],
        [200, 155,  10],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [227, 227, 227]],

       ...,

       [[255, 255, 157],
        [237, 234,  95],
        [235, 234,  32],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 154],
        [233, 232,  90],
        [238, 239,  39],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 150],
        [232, 231,  87],
        [237, 239,  41],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [186]:
with open('output/encoded_image.BMP', 'wb') as f:
    # encode BMP image
    Image.fromarray(image).save(f, 'BMP')

In [187]:
code_encode = "une rune d'eau : 5739. BTW j'adore les 100Hz wink wink"
# convert code to binary string forcing it to be 8 bits long
caracters_values = [ord(c) for c in code_encode]
code_string = ''.join([format(c, '08b') for c in caracters_values])
print(code_string)
code_string = code_string +  format(4, '08b')
code_string

011101010110111001100101001000000111001001110101011011100110010100100000011001000010011101100101011000010111010100100000001110100010000000110101001101110011001100111001001011100010000001000010010101000101011100100000011010100010011101100001011001000110111101110010011001010010000001101100011001010111001100100000001100010011000000110000010010000111101000100000011101110110100101101110011010110010000001110111011010010110111001101011


'01110101011011100110010100100000011100100111010101101110011001010010000001100100001001110110010101100001011101010010000000111010001000000011010100110111001100110011100100101110001000000100001001010100010101110010000001101010001001110110000101100100011011110111001001100101001000000110110001100101011100110010000000110001001100000011000001001000011110100010000001110111011010010110111001101011001000000111011101101001011011100110101100000100'

In [188]:
offset = 0
code_string = "".join(["0" for i in range(offset)]) + code_string

In [189]:
encoding_length = image.shape[0] * image.shape[1] * image.shape[2]
print("encoding space 3 channels ", encoding_length)
print("encoding space 1 channel ", encoding_length // image.shape[2])
print("code length ", len(code_string))
assert len(code_string) <= encoding_length // image.shape[2]

encoding space 3 channels  1035000
encoding space 1 channel  345000
code length  440


In [190]:
def encode_bit(bit, pixel_value):
    if bit == 1:
        if pixel_value % 2 == 0:
            return pixel_value + 1
        elif pixel_value == 255:
            return pixel_value - 1
        else:
            return pixel_value
    else:
        if pixel_value % 2 == 0:
            return pixel_value
        elif pixel_value == 0:
            return pixel_value + 1
        else:
            return pixel_value - 1


In [191]:
# for i, bit in enumerate(code_string):
#     pixel = image[i% image.shape[1], i//image.shape[1] , 0]
#     print(pixel, bit, encode_bit(int(bit), pixel))

In [192]:

for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        if i * image.shape[1] + j < len(code_string):
            for k in range(image.shape[2]):
    
                image[i, j, k] = encode_bit(int(code_string[i * image.shape[1] + j]), image[i, j, k])
        else:
            for k in range(image.shape[2]):
                image[i, j, k] = encode_bit(0, image[i, j, k])

with open('output/encoded_image.bmp', 'wb') as f:
    # encode BMP image
    Image.fromarray(image).save(f, 'BMP')

In [193]:
import re
def decode_bit(pixel_value):
    return pixel_value % 2


def decode_image(image):
    k = 0
    code_string = ''
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            code_string += str(decode_bit(image[i, j, k]))
    return code_string

def to_string(decoded_bits):
    caracters_values = [decoded_bits[i:i + 8] for i in range(0, len(decoded_bits), 8)]
    return ''.join([chr(int(c, 2)) for c in caracters_values])
    

In [194]:
decoded_bits = decode_image(image)
decoded_string = to_string(decoded_bits)
decoded_string.strip("\x00"), len(decoded_string)

("une rune d'eau : 5739. BTW j'adop", 43125)

In [195]:
code_string

'01110101011011100110010100100000011100100111010101101110011001010010000001100100001001110110010101100001011101010010000000111010001000000011010100110111001100110011100100101110001000000100001001010100010101110010000001101010001001110110000101100100011011110111001001100101001000000110110001100101011100110010000000110001001100000011000001001000011110100010000001110111011010010110111001101011001000000111011101101001011011100110101100000100'

In [196]:
loaded_image = imread('output/encoded_image.bmp')
loaded_image.shape

(690, 500, 3)

In [197]:
decoded_bits = decode_image(image)
decoded_string = to_string(decoded_bits)
decoded_string.strip("\x00"), len(decoded_string)

("une rune d'eau : 5739. BTW j'adop", 43125)